In [ ]:
import json
import pandas as pd
import numpy as np
import math as math
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn.model_selection as model_selection
import sklearn.tree as tree
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
import time
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeRegressor
import catboost
from catboost import CatBoostClassifier
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV


In [ ]:
def describe_column(meta):
    """
    Utility function for describing a dataset column (see below for usage)
    """
    def f(x):
        d = pd.Series(name=x.name, dtype=object)
        m = next(m for m in meta if m['name'] == x.name)
        d['Type'] = m['type']
        d['#NaN'] = x.isna().sum()
        d['Description'] = m['desc']
        if m['type'] == 'categorical':
            counts = x.dropna().map(dict(enumerate(m['cats']))).value_counts().sort_index()
            d['Statistics'] = ', '.join(f'{c}({n})' for c, n in counts.items())
        elif m['type'] == 'real' or m['type'] == 'integer':
            stats = x.dropna().agg(['mean', 'std', 'min', 'max'])
            d['Statistics'] = ', '.join(f'{s}={v :.1f}' for s, v in stats.items())
        elif m['type'] == 'boolean':
            counts = x.dropna().astype(bool).value_counts().sort_index()
            d['Statistics'] = ', '.join(f'{c}({n})' for c, n in counts.items())
        else:
            d['Statistics'] = f'#unique={x.nunique()}'
        return d
    return f

def describe_data(data, meta):
    desc = data.apply(describe_column(meta)).T
    desc = desc.style.set_properties(**{'text-align': 'left'})
    desc = desc.set_table_styles([ dict(selector='th', props=[('text-align', 'left')])])
    return desc 

In [ ]:
df_train = pd.read_csv('Training_clean.csv')
def root_mean_squared_log_error(y_true, y_pred):
    # Alternatively: sklearn.metrics.mean_squared_log_error(y_true, y_pred) ** 0.5
    assert (y_true >= 0).all() 
    assert (y_pred >= 0).all()
    log_error = np.log1p(y_pred) - np.log1p(y_true)  # Note: log1p(x) = log(1 + x)
    return np.mean(log_error ** 2) ** 0.5

df_test = pd.read_csv('Test_clean.csv')

In [ ]:
#y1 = df_train.price.values
#X1 = df_train.drop(['id','price', 'street', 'address'], axis=1)
#X1_test = df_test.drop(['id', 'street', 'address'], axis=1)
categorical_features_indices = np.where(X1.dtypes != np.float64)[0]
categorical_features_indices = np.where(X1_test.dtypes != np.float64)[0]

In [ ]:
df_train['price/area'] = df_train['price']/df_train['area_total']
y1 = df_train['price/area'].values
X1 = df_train.drop(['id','price', 'street', 'address', 'area_total'], axis=1)
X1_test = df_test.drop(['id', 'street', 'address', 'area_total'], axis=1)

In [ ]:
#X_train, X_val, y_train, y_val = train_test_split(X1, y1, test_size=0.2, random_state=1, shuffle=True)
#X_train.shape, X_val.shape, y_train.shape,y_val.shape, X1_test.shape

In [ ]:
model_CBR = CatBoostRegressor()

In [ ]:
parameters = {'depth'         : [6,8,10],
                  'learning_rate' : [0.005, 0.01, 0.05],
                  'iterations'    : [1500,2500,3500]
                 }

In [ ]:
grid = GridSearchCV(estimator=model_CBR, param_grid = parameters, cv = 5, n_jobs=-1)
grid.fit(X1, y1)
print(" Results from Grid Search " )
print("\n The best estimator across ALL searched params:\n", grid.best_estimator_)
print("\n The best score across ALL searched params:\n", grid.best_score_)
print("\n The best parameters across ALL searched params:\n", grid.best_params_)

KeyboardInterrupt: 

In [ ]:
regressor = CatBoostRegressor(depth= 8, iterations= 5000, learning_rate= 0.05)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X1, y1, test_size=0.2, random_state=1, shuffle=True)
X_train.shape, X_val.shape, y_train.shape,y_val.shape, X1_test.shape

((18628, 23), (4657, 23), (18628,), (4657,), (9937, 22))

In [ ]:
pip install ipywidgets

     |████████████████████████████████| 121 kB 27.4 MB/s 
     |████████████████████████████████| 1.6 MB 14.2 MB/s 
     |████████████████████████████████| 243 kB 52.1 MB/s 
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
regressor.fit(X_train, y_train,cat_features=categorical_features_indices,eval_set=(X_val, y_val),plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

4006:	learn: 730.3263037	test: 26103.1485408	best: 26101.7463588 (3287)	total: 11m 9s	remaining: 2m 45s
4007:	learn: 729.9069150	test: 26103.2082876	best: 26101.7463588 (3287)	total: 11m 9s	remaining: 2m 45s
4008:	learn: 729.8008650	test: 26103.2248373	best: 26101.7463588 (3287)	total: 11m 9s	remaining: 2m 45s
4009:	learn: 729.7232842	test: 26103.2060313	best: 26101.7463588 (3287)	total: 11m 9s	remaining: 2m 45s
4010:	learn: 729.6260626	test: 26103.2064452	best: 26101.7463588 (3287)	total: 11m 9s	remaining: 2m 45s
4011:	learn: 729.2910226	test: 26103.0122847	best: 26101.7463588 (3287)	total: 11m 9s	remaining: 2m 44s
4012:	learn: 729.1432376	test: 26103.0357633	best: 26101.7463588 (3287)	total: 11m 9s	remaining: 2m 44s
4013:	learn: 728.8962952	test: 26103.0378782	best: 26101.7463588 (3287)	total: 11m 9s	remaining: 2m 44s
4014:	learn: 728.8389541	test: 26103.0372935	best: 26101.7463588 (3287)	total: 11m 10s	remaining: 2m 44s
4015:	learn: 728.6797458	test: 26102.8735224	best: 26101.746358

In [ ]:
print(regressor.score(X_train, y_train))
from sklearn.metrics import r2_score

y_pred = regressor.predict(X_val)
y_pred_train = regressor.predict(X_train)
#y_pred = y_pred.astype(int)
print(regressor.score(X_val, y_val)), print(r2_score(y_pred, regressor.predict(X_val)))

0.9990910572678512
0.976437129991934
1.0


(None, None)

In [ ]:

#y_train_hat = model.predict(X_train)
#y_valid_hat = model.predict(X_valid)
#y_tests = model.predict()
print(f'Train RMSLE: {root_mean_squared_log_error(y_true=y_train, y_pred=y_pred_train) :.4f}')
print(f'Valid RMSLE: {root_mean_squared_log_error(y_true=y_val, y_pred=y_pred) :.4f}')

Train RMSLE: 0.0082
Valid RMSLE: 0.0180


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=99c1e030-46bc-40fa-835c-0661eae0d488' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>